In [2]:
! pip install librosa
! pip install sounddevice

In [3]:
import librosa
import numpy as np
import sounddevice as sd

In [4]:
def speak_text(text, voice_profile):
    text_splits = text.split(" ")
    print(text_splits)
    
    concatenated_audio = np.array([])
    for i in range(len(text_splits)):
        clip_path = "clips/" + voice_profile + "/" + text_splits[i] + ".wav"
        audio, sr = librosa.load(clip_path, sr=8000)
        concatenated_audio = np.concatenate([concatenated_audio, audio])
    
    sd.play(concatenated_audio, 8000)
    sd.wait()


def speak_text_multiple(text_array, voice_profile):
    grand_concatenated_audio = np.array([])
    for i in range(len(text_array)):
        text = text_array[i]
    
        text_splits = text.split(" ")
        
        concatenated_audio = np.array([])
        for i in range(len(text_splits)):
            clip_path = "clips/" + voice_profile + "/" + text_splits[i] + ".wav"
            audio, sr = librosa.load(clip_path, sr=8000)
            concatenated_audio = np.concatenate([concatenated_audio, audio])
    
        grand_concatenated_audio = np.concatenate([grand_concatenated_audio, np.array([point for point in range(int(0.5 * 8000))]), concatenated_audio])
    
    sd.play(grand_concatenated_audio, 8000)
    sd.wait()

In [8]:
text = "ball is not cat and cat is not ball"
voice_profile = "pradeep"
speak_text(text, voice_profile)

['ball', 'is', 'not', 'cat', 'and', 'cat', 'is', 'not', 'ball']


In [10]:
text_array = [
    "this is a cat", 
    "that is a ball", 
    "cat is not ball", 
    "this cat is not that ball", 
    "ball is not cat and cat is not ball",
    "a ball is not a cat and that is"
]
voice_profile = "sahil" 
speak_text_multiple(text_array, voice_profile)